In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


# NOTE: Compared to version 1, This version uses KNN to impute data.

In [7]:
train = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv', index_col=['id'])
test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv', index_col=['id'])
train_xtra = pd.read_csv('/kaggle/input/playground-series-s5e2/training_extra.csv', index_col=['id'])

train_xtra

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
500000,Under Armour,Canvas,Small,10.0,Yes,Yes,Tote,Blue,23.882052,114.11068
500001,Puma,Polyester,Small,4.0,No,Yes,Backpack,Green,11.869095,129.74972
500002,Jansport,Polyester,Small,8.0,Yes,Yes,Tote,Red,8.092302,21.37370
500003,Nike,Nylon,Large,7.0,No,No,Messenger,Pink,7.719581,48.09209
500004,Nike,Leather,Large,9.0,No,Yes,Tote,Green,22.741826,77.32461
...,...,...,...,...,...,...,...,...,...,...
4194313,Nike,Canvas,NaN,3.0,Yes,Yes,Messenger,Blue,28.098120,104.74460
4194314,Puma,Leather,Small,10.0,Yes,Yes,Tote,Blue,17.379531,122.39043
4194315,Jansport,Canvas,Large,10.0,No,No,Backpack,Red,17.037708,148.18470


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300000 entries, 0 to 299999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Brand                 290295 non-null  object 
 1   Material              291653 non-null  object 
 2   Size                  293405 non-null  object 
 3   Compartments          300000 non-null  float64
 4   Laptop Compartment    292556 non-null  object 
 5   Waterproof            292950 non-null  object 
 6   Style                 292030 non-null  object 
 7   Color                 290050 non-null  object 
 8   Weight Capacity (kg)  299862 non-null  float64
 9   Price                 300000 non-null  float64
dtypes: float64(3), object(7)
memory usage: 25.2+ MB


*Create a pairplot*

*Get numeric columns and categorical columns*

In [9]:
num_df = train.select_dtypes(include=['number']).drop('Price', axis=1)
cat_df = train.select_dtypes(include=['object'])

num_cols = num_df.columns
cat_cols = cat_df.columns

num_df

,Compartments,Weight Capacity (kg)
id,,
0,7.0,11.611723
1,10.0,27.078537
2,2.0,16.643760
3,8.0,12.937220
4,1.0,17.749338
...,...,...
299995,9.0,12.730812
299996,6.0,26.633182
299997,9.0,11.898250


*Seperate categorical columns into ordinal (to be ordinal encoded) and non-ordinal (to be one-hot encoded)*

# TODO: Try changing the ordinal and non-ordinal columns around

In [10]:
ord_cols = ['Size', 'Laptop Compartment', 'Waterproof']
oh_cols = ['Brand', 'Material', 'Style', 'Color']

ord_df = train[ord_cols]
oh_df = train[oh_cols]

*Get unique values*

In [11]:
for col in ord_df.columns:
    print(f'col: {col}\n\
values: {ord_df[col].unique()}')

col: Size
values: ['Medium' 'Small' 'Large' nan]
col: Laptop Compartment
values: ['Yes' 'No' nan]
col: Waterproof
values: ['No' 'Yes' nan]


*Get the ordering of columns in the `train` DataFrame*

In [12]:
for i, col in enumerate(train.columns):
    print(f'{i}, {col}')

0, Brand
1, Material
2, Size
3, Compartments
4, Laptop Compartment
5, Waterproof
6, Style
7, Color
8, Weight Capacity (kg)
9, Price


 *Define ordering for each value*

In [13]:
size_categories = ['Small', 'Medium', 'Large']
yesno_categories = ['No', 'Yes']

*Create transformers for each type of column*

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer

# Using KNNImputer, n_neighbors is set arbitrarily, we'll go back later and figure out the best setting

# Numerical columns

num_transformer = Pipeline(
    steps=[
        ('num_impute', KNNImputer(n_neighbors = 2)),
        ('scale', StandardScaler()),
    ]
)

num_df = pd.DataFrame(num_transformer.fit_transform(num_df))
num_df.columns = num_cols

# For ordinal columns

ord_preprocessor = ColumnTransformer(
    transformers=[
        ('size_encoder', OrdinalEncoder(categories=[size_categories], ), ['Size']),
        ('laptop_encoder', OrdinalEncoder(categories=[yesno_categories]), ['Laptop Compartment']),
        ('wp_encoder', OrdinalEncoder(categories=[yesno_categories]), ['Waterproof'])
    ],
    remainder='passthrough'
)

# We have to use function transformer because `ord_impute` converts the DataFrame to a ndarray
# In line with best practices using `ord_preprocessor`, we call columns by name, so we use
# function transformer to convert our ndarray back to a DataFrame

ord_transformer = Pipeline(
    steps=[
        ('ord_impute', SimpleImputer(strategy='most_frequent')),
        ('df_conversion', FunctionTransformer(lambda x: pd.DataFrame(x, columns=ord_cols))),
        ('preprocess', ord_preprocessor)
    ]
)

ord_df = pd.DataFrame(ord_transformer.fit_transform(ord_df), columns = ord_cols)

# One hot columns

oh_transformer = Pipeline(
    steps=[
        ('oh_impute', SimpleImputer(strategy='most_frequent')),
        ('oh_encode', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ]
)

oh_df = pd.DataFrame(oh_transformer.fit_transform(oh_df))



ValueError: could not convert string to float: 'Small'

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

my_model = xgb.XGBRegressor(objective='reg:squarederror',
                        n_estimators=500)

preprocessor = ColumnTransformer(
    transformers = [
        ('num', num_transformer, num_cols),
        ('ord', ord_transformer, ord_cols),
        ('oh', oh_transformer, oh_cols)
    ],
    remainder = 'passthrough'
)

pipeline = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('predict', my_model)
    ]
)

train_total = pd.concat([train, train_xtra], axis = 0)
train_total

X = train_total.drop(columns = ['Price'])
y = train_total['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.fit_transform(X_test)
my_model.fit(X_train, y_train,
            early_stopping_rounds = 5,
            eval_set=[(X_test, y_test)])

In [ ]:
my_model.save_model("xgboost_model.json") 

Not sure if below is the same model?

In [ ]:
# import xgboost as xgb

# loaded_model = xgb.XGBRegressor()

# loaded_model.load_model("xgboost_model.json")
# loaded_model

In [ ]:
predictions = my_model.predict(preprocessor.fit_transform(test))

In [ ]:
submission = pd.DataFrame({'Price': predictions})
submission = submission.set_index(test.index)
submission

In [ ]:
submission.to_csv('submission.csv')